# ORES API Example
This example illustrates how to generate quality scores for article revisions using [ORES](https://www.mediawiki.org/wiki/ORES). This example shows how to request a score of a specific revision, where the score provides probabilities for all of the possible article quality levels. The API documentation can be access from the main [ORES](https://ores.wikimedia.org) page. However, this documentation is a little skimpy and if you want more information you may have to dig around.

## License
This code example was developed by Dr. David W. McDonald for use in DATA 512, a course in the UW MS Data Science degree program. This code is provided under the [Creative Commons](https://creativecommons.org) [CC-BY license](https://creativecommons.org/licenses/by/4.0/). Revision 1.0 - May 13, 2022



In [1]:
# 
# These are standard python modules
import json, time, urllib.parse
#
# The 'requests' module is not a standard Python module. You will need to install this with pip/pip3 if you do not already have it
import requests

The example relies on some constants that help make the code a bit more readable.

In [2]:
#########
#
#    CONSTANTS
#

# The current ORES API endpoint
API_ORES_SCORE_ENDPOINT = "https://ores.wikimedia.org/v3"
# A template for mapping to the URL
API_ORES_SCORE_PARAMS = "/scores/{context}/{revid}/{model}"

# Use some delays so that we do not hammer the API with our requests
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = (1.0/100.0)-API_LATENCY_ASSUMED

# When making automated requests we should include something that is unique to the person making the request
# This should include an email - your UW email would be good to put in there
REQUEST_HEADERS = {
    'User-Agent': '<uwnetid@uw.edu>, University of Washington, MSDS DATA 512 - AUTUMN 2022'
}

# A dictionary of English Wikipedia article titles (keys) and sample revision IDs that can be used for this ORES scoring example
# ARTICLE_REVISIONS = { 'Bison':1085687913 , 'Northern flicker':1086582504 , 'Red squirrel':1083787665 , 'Chinook salmon':1085406228 , 'Horseshoe bat':1060601936 }

# This template lists the basic parameters for making an ORES request
ORES_PARAMS_TEMPLATE = {
    "context": "enwiki",        # which WMF project for the specified revid
    "revid" : "",               # the revision to be scored - this will probably change each call
    "model": "articlequality"   # the AI/ML scoring model to apply to the reviewion
}
#
# The current ML models for English wikipedia are:
#   "articlequality"
#   "articletopic"
#   "damaging"
#   "version"
#   "draftquality"
#   "drafttopic"
#   "goodfaith"
#   "wp10"
#
# The specific documentation on these is scattered so if you want to use one you'll have to look around.
#

The API request will be made using one procedure. The idea is to make this reusable. The procedure is parameterized, but relies on the constants above for the important parameters. The underlying assumption is that this will be used to request data for a set of article revisions. Therefore, the main parameter is article_revid.

In [18]:
#------Load json with title and LastRevID ----#
import json
import pandas as pd

def create_article_revisions(file, column):
    output_df = pd.DataFrame()
    f = open(file)
    df = json.load(f)
    
    for i in range(0, len(df)-1):
        temp_dict = df[i][column]
        pull_df = pd.DataFrame.from_dict(temp_dict, orient = 'index')
        pull_df = pull_df.T
        output_df = pd.concat([output_df, pull_df])
    return output_df

In [4]:
df_with_articles = create_article_revisions("politicians_by_country.json", 'pages')
df_with_articles

,pageid,ns,title,contentmodel,pagelanguage,pagelanguagehtmlcode,pagelanguagedir,touched,lastrevid,length,talkid,fullurl,editurl,canonicalurl,watchers,redirect,new,missing
0,42972519,0,Abdul Ghafar Lakanwal,wikitext,en,en,ltr,2022-09-26T05:36:04Z,943562276,4165,42972696,https://en.wikipedia.org/wiki/Abdul_Ghafar_Lak...,https://en.wikipedia.org/w/index.php?title=Abd...,https://en.wikipedia.org/wiki/Abdul_Ghafar_Lak...,NaN,NaN,NaN,NaN
0,10483286,0,Majah Ha Adrif,wikitext,en,en,ltr,2022-10-10T00:16:18Z,852404094,3162,13330265,https://en.wikipedia.org/wiki/Majah_Ha_Adrif,https://en.wikipedia.org/w/index.php?title=Maj...,https://en.wikipedia.org/wiki/Majah_Ha_Adrif,NaN,NaN,NaN,NaN
0,11966231,0,Haroon al-Afghani,wikitext,en,en,ltr,2022-10-10T02:53:39Z,1095102390,16718,15250816,https://en.wikipedia.org/wiki/Haroon_al-Afghani,https://en.wikipedia.org/w/index.php?title=Har...,https://en.wikipedia.org/wiki/Haroon_al-Afghani,NaN,NaN,NaN,NaN
0,46841383,0,Tayyab Agha,wikitext,en,en,ltr,2022-10-05T14:31:44Z,1104998382,6313,46843786,https://en.wikipedia.org/wiki/Tayyab_Agha,https://en.wikipedia.org/w/index.php?title=Tay...,https://en.wikipedia.org/wiki/Tayyab_Agha,NaN,NaN,NaN,NaN
0,68624823,0,Ahmadullah Wasiq,wikitext,en,en,ltr,2022-10-05T14:54:20Z,1109361754,5267,68639469,https://en.wikipedia.org/wiki/Ahmadullah_Wasiq,https://en.wikipedia.org/w/index.php?title=Ahm...,https://en.wikipedia.org/wiki/Ahmadullah_Wasiq,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,10031005,0,Gift Tandare,wikitext,en,en,ltr,2022-10-07T14:38:25Z,1114641622,767,10495324,https://en.wikipedia.org/wiki/Gift_Tandare,https://en.wikipedia.org/w/index.php?title=Gif...,https://en.wikipedia.org/wiki/Gift_Tandare,NaN,NaN,NaN,NaN
0,23231242,0,Rekayi Tangwena,wikitext,en,en,ltr,2022-09-26T04:15:54Z,1073818982,1861,23734031,https://en.wikipedia.org/wiki/Rekayi_Tangwena,https://en.wikipedia.org/w/index.php?title=Rek...,https://en.wikipedia.org/wiki/Rekayi_Tangwena,NaN,NaN,NaN,NaN
0,633594,0,Josiah Tongogara,wikitext,en,en,ltr,2022-09-25T23:01:07Z,1106932400,12797,3349627,https://en.wikipedia.org/wiki/Josiah_Tongogara,https://en.wikipedia.org/w/index.php?title=Jos...,https://en.wikipedia.org/wiki/Josiah_Tongogara,NaN,NaN,NaN,NaN
0,16375315,0,Langton Towungana,wikitext,en,en,ltr,2022-09-19T23:24:46Z,904246837,1498,16375344,https://en.wikipedia.org/wiki/Langton_Towungana,https://en.wikipedia.org/w/index.php?title=Lan...,https://en.wikipedia.org/wiki/Langton_Towungana,NaN,NaN,NaN,NaN


In [5]:
#########
#
#    PROCEDURES/FUNCTIONS
#

def request_ores_score_per_article(article_revid = None, 
                                   endpoint_url = API_ORES_SCORE_ENDPOINT, 
                                   endpoint_params = API_ORES_SCORE_PARAMS, 
                                   request_template = ORES_PARAMS_TEMPLATE,
                                   headers = REQUEST_HEADERS,
                                   features=False):
    # Make sure we have an article revision id
    if not article_revid: return None
    
    # set the revision id into the template
    request_template['revid'] = article_revid
    
    # now, create a request URL by combining the endpoint_url with the parameters for the request
    request_url = endpoint_url+endpoint_params.format(**request_template)
    
    # the features used by the ML model can sometimes be returned as well as scores
    if features:
        request_url = request_url+"?features=true"
    
    # make the request
    try:
        # we'll wait first, to make sure we don't exceed the limit in the situation where an exception
        # occurs during the request processing - throttling is always a good practice with a free
        # data source like ORES - or other community sources
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(request_url, headers=headers)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response


In [6]:
df_with_articles['lastrevid'].iloc[0]

943562276

In [7]:
def create_dataframe(filename, df):
    output_df = pd.DataFrame()
    for i in range(1, len(df)-1):
        print("Getting pageview data for: ",df['title'].iloc[i])
        print("Percentage done: ", i/len(df))
        info = request_ores_score_per_article(df['lastrevid'].iloc[i])
        views_df = pd.DataFrame(info)
        output_df = pd.concat([output_df, views_df])

    result = output_df.to_json(orient="records")
    parsed = json.loads(result)
    print(output_df)

    f = open(filename, 'w')
    f.write(json.dumps(parsed, indent=4))
    f.close()
    
    print("Wrote to file!")
    return output_df
    

In [8]:
val_df = create_dataframe('scores.json', df_with_articles)

Getting pageview data for:  Majah Ha Adrif
Percentage done:  0.00013189132155104195
Getting pageview data for:  Haroon al-Afghani
Percentage done:  0.0002637826431020839
Getting pageview data for:  Tayyab Agha
Percentage done:  0.0003956739646531258
Getting pageview data for:  Ahmadullah Wasiq
Percentage done:  0.0005275652862041678
Getting pageview data for:  Aziza Ahmadyar
Percentage done:  0.0006594566077552098
Getting pageview data for:  Muqadasa Ahmadzai
Percentage done:  0.0007913479293062516
Getting pageview data for:  Mohammad Sarwar Ahmedzai
Percentage done:  0.0009232392508572936
Getting pageview data for:  Amir Muhammad Akhundzada
Percentage done:  0.0010551305724083356
Getting pageview data for:  Nasrullah Baryalai Arsalai
Percentage done:  0.0011870218939593776
Getting pageview data for:  Mohammad Asim Asim
Percentage done:  0.0013189132155104195
Getting pageview data for:  Atiqullah Atifmal
Percentage done:  0.0014508045370614613
Getting pageview data for:  Abdul Rahim Ay

Getting pageview data for:  Sayyid Husayn
Percentage done:  0.01292534951200211
Getting pageview data for:  Suhail Shaheen
Percentage done:  0.013057240833553153
Getting pageview data for:  Mohammad Shakir Kargar
Percentage done:  0.013189132155104194
Getting pageview data for:  Baktash Siawash
Percentage done:  0.013321023476655236
Getting pageview data for:  Awtar Singh
Percentage done:  0.013452914798206279
Getting pageview data for:  Hasan Ali Sultani
Percentage done:  0.01358480611975732
Getting pageview data for:  Faramarz Tamanna
Percentage done:  0.013716697441308361
Getting pageview data for:  Mohammad Hashem Taufiqui
Percentage done:  0.013848588762859404
Getting pageview data for:  Nur ul-Haq Ulumi
Percentage done:  0.013980480084410446
Getting pageview data for:  Pacha Gul Wafadar
Percentage done:  0.014112371405961487
Getting pageview data for:  Wahidullah Waissi
Percentage done:  0.01424426272751253
Getting pageview data for:  Gul Ahmmad Yama
Percentage done:  0.014376154

Getting pageview data for:  Menduh Zavalani
Percentage done:  0.026246372988657347
Getting pageview data for:  Said Abadou
Percentage done:  0.02637826431020839
Getting pageview data for:  Tahar Allan
Percentage done:  0.02651015563175943
Getting pageview data for:  Mohamed Seghir Babes
Percentage done:  0.02664204695331047
Getting pageview data for:  Djelloul Baghli
Percentage done:  0.026773938274861513
Getting pageview data for:  Noureddine Bahbouh
Percentage done:  0.026905829596412557
Getting pageview data for:  Lamine Bechichi
Percentage done:  0.0270377209179636
Getting pageview data for:  Bakhti Belaïb
Percentage done:  0.02716961223951464
Getting pageview data for:  Mohamed Belouizdad
Percentage done:  0.02730150356106568
Getting pageview data for:  Mourad Benachenou
Percentage done:  0.027433394882616723
Getting pageview data for:  Benaouda Hadj Hacène Bachterzi
Percentage done:  0.027565286204167767
Getting pageview data for:  Souad Bendjaballah
Percentage done:  0.027697177

Getting pageview data for:  Harold Lovell
Percentage done:  0.03903983117910841
Getting pageview data for:  John Lyons (Antiguan politician)
Percentage done:  0.03917172250065946
Getting pageview data for:  Edmond Mansoor
Percentage done:  0.039303613822210495
Getting pageview data for:  Joanne Massiah
Percentage done:  0.03943550514376154
Getting pageview data for:  Melford Walter Nicholas
Percentage done:  0.039567396465312585
Getting pageview data for:  Bernard Percival
Percentage done:  0.03969928778686362
Getting pageview data for:  Alincia Williams-Grant
Percentage done:  0.03983117910841467
Getting pageview data for:  Joaquín Samuel de Anchorena
Percentage done:  0.039963070429965705
Getting pageview data for:  Juan Agustín Augier
Percentage done:  0.04009496175151675
Getting pageview data for:  Uladislao Augier
Percentage done:  0.040226853073067795
Getting pageview data for:  Joaquín Belgrano
Percentage done:  0.04035874439461883
Getting pageview data for:  Antonio Bermejo
Per

Getting pageview data for:  Serop Zakaryan
Percentage done:  0.05196518069111052
Getting pageview data for:  Servitchen
Percentage done:  0.05209707201266157
Getting pageview data for:  Suren Shadunts
Percentage done:  0.05222896333421261
Getting pageview data for:  Kasbar Sinabian
Percentage done:  0.05236085465576365
Getting pageview data for:  Ohannes Tchekidjian
Percentage done:  0.052492745977314695
Getting pageview data for:  Hrayr Tovmasyan
Percentage done:  0.05262463729886573
Getting pageview data for:  Usuv Beg
Percentage done:  0.05275652862041678
Getting pageview data for:  Kegham Vanigian
Percentage done:  0.052888419941967815
Getting pageview data for:  Artak Zeynalyan
Percentage done:  0.05302031126351886
Getting pageview data for:  Alfons Adam
Percentage done:  0.053152202585069905
Getting pageview data for:  Ingo Appelt
Percentage done:  0.05328409390662094
Getting pageview data for:  Baron Alexander von Bach
Percentage done:  0.05341598522817199
Getting pageview data 

...

Getting pageview data for:  Ssendi Mosh Afrikan
Percentage done:  0.9419678185175415
Getting pageview data for:  Apollo Ofwono
Percentage done:  0.9420997098390926
Getting pageview data for:  Christine Apolot
Percentage done:  0.9422316011606436
Getting pageview data for:  Florence Akiiki Asiimwe
Percentage done:  0.9423634924821946
Getting pageview data for:  Charles Ayume
Percentage done:  0.9424953838037458
Getting pageview data for:  Edith Mary Bataringaya
Percentage done:  0.9426272751252968
Getting pageview data for:  Flavia Rwabuhoro Kabahenda
Percentage done:  0.9427591664468478
Getting pageview data for:  John Kabaireho
Percentage done:  0.9428910577683989
Getting pageview data for:  Joseph Kabuleta
Percentage done:  0.9430229490899499
Getting pageview data for:  Apollo Kaggwa
Percentage done:  0.9431548404115009
Getting pageview data for:  Michael Kawalya Kagwa
Percentage done:  0.943286731733052
Getting pageview data for:  Robert Mugabe Kakyebezi
Percentage done:  0.94341862

Getting pageview data for:  Viacheslav Suprunenko
Percentage done:  0.9552888419941968
Getting pageview data for:  Taras Tarasenko
Percentage done:  0.9554207333157478
Getting pageview data for:  Tamila Tasheva
Percentage done:  0.9555526246372988
Getting pageview data for:  Sergei Timchenko
Percentage done:  0.95568451595885
Getting pageview data for:  Volodymyr Tsvil
Percentage done:  0.955816407280401
Getting pageview data for:  Volodymyr Tyahlo
Percentage done:  0.955948298601952
Getting pageview data for:  Kyrylo Tymoshenko
Percentage done:  0.9560801899235031
Getting pageview data for:  Evgeniy Udod
Percentage done:  0.9562120812450541
Getting pageview data for:  Andriy Veselovsky
Percentage done:  0.9563439725666051
Getting pageview data for:  Dmytro Vorona
Percentage done:  0.9564758638881562
Getting pageview data for:  Mykola Yankovsky
Percentage done:  0.9566077552097072
Getting pageview data for:  Alla Yaroshinskaya
Percentage done:  0.9567396465312582
Getting pageview data 

Getting pageview data for:  Sa'dulla Begaliyev
Percentage done:  0.9679504088630968
Getting pageview data for:  Torokul Dzhanuzakov
Percentage done:  0.9680823001846478
Getting pageview data for:  Rustam Inoyatov
Percentage done:  0.9682141915061989
Getting pageview data for:  Rashid Kadyrov
Percentage done:  0.9683460828277499
Getting pageview data for:  Karim Kamalov
Percentage done:  0.9684779741493009
Getting pageview data for:  Tatyana Karimova
Percentage done:  0.968609865470852
Getting pageview data for:  Bakhodir Kurbanov (general)
Percentage done:  0.968741756792403
Getting pageview data for:  Jahangir Mamatov
Percentage done:  0.9688736481139542
Getting pageview data for:  Boʻritosh Mustafoyev
Percentage done:  0.9690055394355052
Getting pageview data for:  Narzullo Oblomurodov
Percentage done:  0.9691374307570562
Getting pageview data for:  Baxtiyor Rahimov
Percentage done:  0.9692693220786073
Getting pageview data for:  Bekjon Rakhmonov
Percentage done:  0.9694012134001583


Getting pageview data for:  Nguyễn Văn Lực
Percentage done:  0.98100764969665
Getting pageview data for:  Nguyễn Văn Trân
Percentage done:  0.981139541018201
Getting pageview data for:  Nguyễn Văn Xuân
Percentage done:  0.981271432339752
Getting pageview data for:  Tran Minh Tiet
Percentage done:  0.9814033236613031
Getting pageview data for:  Trần Quốc Vượng (politician)
Percentage done:  0.9815352149828541
Getting pageview data for:  Trần Tuấn Anh
Percentage done:  0.9816671063044051
Getting pageview data for:  Trương Đình Dzu
Percentage done:  0.9817989976259562
Getting pageview data for:  Võ Văn Thưởng
Percentage done:  0.9819308889475072
Getting pageview data for:  Abdullah al-Alimi Bawazeer
Percentage done:  0.9820627802690582
Getting pageview data for:  Abdullah Badheeb
Percentage done:  0.9821946715906094
Getting pageview data for:  Fathi Tawfiq Abdulrahim
Percentage done:  0.9823265629121604
Getting pageview data for:  Hamid al-Ahmar
Percentage done:  0.9824584542337115
Gettin

Getting pageview data for:  Josiah Zion Gumede
Percentage done:  0.9940648905302031
Getting pageview data for:  Ellen Gwaradzimba
Percentage done:  0.9941967818517542
Getting pageview data for:  Andre S Holland
Percentage done:  0.9943286731733052
Getting pageview data for:  Byron Hove
Percentage done:  0.9944605644948562
Getting pageview data for:  Tonderai Kasu
Percentage done:  0.9945924558164073
Getting pageview data for:  Wilson Khumbula
Percentage done:  0.9947243471379583
Getting pageview data for:  Lovemore Madhuku
Percentage done:  0.9948562384595093
Getting pageview data for:  Sarah Mahoka
Percentage done:  0.9949881297810604
Getting pageview data for:  Alfred Makwarimba
Percentage done:  0.9951200211026114
Getting pageview data for:  Washington Malianga
Percentage done:  0.9952519124241624
Getting pageview data for:  Elliot Manyika
Percentage done:  0.9953838037457136
Getting pageview data for:  Linda Masarira
Percentage done:  0.9955156950672646
Getting pageview data for:  

In [21]:
df_with_scores = create_article_revisions('scores.json', 'enwiki')
df_with_scores

,articlequality,852404094
version,0.9.2,NaN
articlequality,NaN,"{'score': {'prediction': 'Start', 'probability..."
version,0.9.2,NaN
